### Библиотеки

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gmean

import statsmodels.api as sm
from scipy.stats import t, f, boxcox, skew, kurtosis
from statsmodels.stats.diagnostic import linear_reset, het_white


import warnings
warnings.filterwarnings('ignore')

### Обработка

In [50]:
data = pd.read_csv('data_after_processing.csv', encoding='utf-8',)

data = data.drop(['title'], axis=1)
data = data.drop(['author_Другой', 'publisher_Другой', 'publication_year_Другой',
                  'cover_type_Мягкий заламинированный картон', 'reading_age_6+'], axis=1)

### Логарифм цены

In [51]:
data['log_price'] = np.log(data['price'])

In [52]:
data.columns

Index(['price', 'avg_rating', 'cnt_reviews', 'pages_cnt', 'tirage', 'weight',
       'author_Джейн Остен', 'author_Джек Лондон', 'author_Джордж Оруэлл',
       'author_Лев Толстой', 'author_Луиза Мэй Олкотт',
       'author_Михаил Булгаков', 'author_Николай Гоголь',
       'author_Федор Достоевский', 'author_Эрих Ремарк', 'publisher_АСТ',
       'publisher_Азбука', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'cover_type_Мягкий переплёт',
       'cover_type_Твёрдый переплёт', 'reading_age_0+', 'reading_age_12+',
       'reading_age_16+', 'reading_age_18+', 'thickness', 'width', 'length',
       'volume', 'log_price'],
      dtype='object')

### Регрессоры, которые можно логарифмировать и нет

In [59]:
unconditional_cols = [
    'author_Джейн Остен', 'author_Джек Лондон', 'author_Джордж Оруэлл',
       'author_Лев Толстой', 'author_Луиза Мэй Олкотт',
       'author_Михаил Булгаков', 'author_Николай Гоголь',
       'author_Федор Достоевский', 'author_Эрих Ремарк', 'publisher_АСТ',
       'publisher_Азбука', 'publisher_Иностранка',
       'publisher_Манн, Иванов и Фербер', 'publisher_Эксмо',
       'publication_year_2021', 'publication_year_2022',
       'publication_year_2023', 'publication_year_2024',
       'publication_year_2025', 'cover_type_Мягкий переплёт',
       'cover_type_Твёрдый переплёт', 'reading_age_0+', 'reading_age_12+',
       'reading_age_16+', 'reading_age_18+',
]

cols_to_try_log = [
    'avg_rating', 'cnt_reviews', 'pages_cnt', 'tirage', 
    'weight', 'thickness', 'width', 'length', 'volume'
]

### Подбор функциональной формы для $\log{price}$

In [60]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from itertools import product
from tqdm import tqdm  # для прогресс-бара (опционально)


def generate_all_combinations():
    """Генерирует все комбинации логарифмирования для cols_to_try_log"""
    return product([False, True], repeat=len(cols_to_try_log))

def prepare_X(combination, df):
    """Создает матрицу X для текущей комбинации"""
    X_temp = df[unconditional_cols].copy()
    
    for i, col in enumerate(cols_to_try_log):
        if combination[i]:
            # Проверка на положительные значения
            if (df[col] <= 0).any():
                # Добавляем константу, чтобы избежать log(0)
                X_temp[f'log_{col}'] = np.log(df[col] + 1e-6)
            else:
                X_temp[f'log_{col}'] = np.log(df[col])
        else:
            X_temp[col] = df[col]
    
    return sm.add_constant(X_temp)

results = []

# Генерируем все комбинации (2^N вариантов)
total_combinations = 2 ** len(cols_to_try_log)

for combination in tqdm(generate_all_combinations(), total=total_combinations):
    try:
        # Подготавливаем данные
        X_curr = prepare_X(combination, data)
        
        # # Удаляем возможные NaN
        # X_curr_clean = X_curr.dropna(axis=1, how='any')
        # y_clean = y.reindex_like(X_curr_clean)
        
        # Строим модель
        model = sm.OLS(y, X_curr).fit()
        
        # Сохраняем результаты
        results.append({
            'combination': combination,
            'aic': model.aic,
            'bic': model.bic,
            'adj_r2': model.rsquared_adj,
            'model': model
        })
    
    except Exception as e:
        print(f"Error in combination {combination}: {str(e)}")
        continue

# Создаем DataFrame с результатами
df_results = pd.DataFrame(results)

# Добавляем расшифровку комбинаций
df_results['log_columns'] = df_results['combination'].apply(
    lambda x: [cols_to_try_log[i] for i, log in enumerate(x) if log]
)

# Находим лучшие модели по разным критериям
best_aic = df_results.loc[df_results['aic'].idxmin()]
best_bic = df_results.loc[df_results['bic'].idxmin()]
best_adj_r2 = df_results.loc[df_results['adj_r2'].idxmax()]

print("Лучшая модель по AIC:")
print(f"Логарифмированные переменные: {best_aic['log_columns']}")
print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Логарифмированные переменные: {best_bic['log_columns']}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Логарифмированные переменные: {best_adj_r2['log_columns']}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")


100%|██████████| 512/512 [00:05<00:00, 93.00it/s] 


Лучшая модель по AIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
AIC: -394.16

Лучшая модель по BIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
BIC: -181.07

Лучшая модель по Adj.R²:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'weight', 'thickness']
Adj.R²: 0.8077


### Подбор функциональной формы для $price$

In [61]:
results = []

# Генерируем все комбинации (2^N вариантов)
total_combinations = 2 ** len(cols_to_try_log)

for combination in tqdm(generate_all_combinations(), total=total_combinations):
    try:
        # Подготавливаем данные
        X_curr = prepare_X(combination, data)
        
        # # Удаляем возможные NaN
        # X_curr_clean = X_curr.dropna(axis=1, how='any')
        # y_clean = y.reindex_like(X_curr_clean)
        
        # Строим модель
        model = sm.OLS(data['price'], X_curr).fit()
        
        # Сохраняем результаты
        results.append({
            'combination': combination,
            'aic': model.aic,
            'bic': model.bic,
            'adj_r2': model.rsquared_adj,
            'model': model
        })
    
    except Exception as e:
        print(f"Error in combination {combination}: {str(e)}")
        continue

# Создаем DataFrame с результатами
df_results = pd.DataFrame(results)

# Добавляем расшифровку комбинаций
df_results['log_columns'] = df_results['combination'].apply(
    lambda x: [cols_to_try_log[i] for i, log in enumerate(x) if log]
)

# Находим лучшие модели по разным критериям
best_aic = df_results.loc[df_results['aic'].idxmin()]
best_bic = df_results.loc[df_results['bic'].idxmin()]
best_adj_r2 = df_results.loc[df_results['adj_r2'].idxmax()]

print("Лучшая модель по AIC:")
print(f"Логарифмированные переменные: {best_aic['log_columns']}")
print(f"AIC: {best_aic['aic']:.2f}\n")

print("Лучшая модель по BIC:")
print(f"Логарифмированные переменные: {best_bic['log_columns']}")
print(f"BIC: {best_bic['bic']:.2f}\n")

print("Лучшая модель по Adj.R²:")
print(f"Логарифмированные переменные: {best_adj_r2['log_columns']}")
print(f"Adj.R²: {best_adj_r2['adj_r2']:.4f}")


100%|██████████| 512/512 [00:06<00:00, 75.19it/s] 


Лучшая модель по AIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'width', 'volume']
AIC: 41424.55

Лучшая модель по BIC:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'width', 'volume']
BIC: 41637.64

Лучшая модель по Adj.R²:
Логарифмированные переменные: ['pages_cnt', 'tirage', 'width', 'volume']
Adj.R²: 0.8541


### Вывод

Нужно логарифмировать ['pages_cnt', 'tirage', 'weight', 'thickness']

Все переборы и для lnY, и для Y показывают один и тот же результат